# MBA em Ciência de Dados
# Técnicas Avançadas de Captura e Tratamento de Dados

### <span style="color:darkred">Módulo II - Tratamento e limpeza de Dados</span>


### <span style="color:darkred">Avaliação</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [1]:
# carregando as bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn import metrics

# carregando dados
# data = pd.read_csv("./dados/pib_mba_avaliacao2021.csv")
data = pd.read_csv("pib_mba_avaliacao2021.csv")

Vamos utilizar uma base de dados baseada em informações de população e PIB para cada município brasileiro, essa base foi adaptada da original do IBGE para o propósito dessa avaliação (e portanto parte das variáveis **estão modificadas e não são reais**). A base possui as seguintes colunas:
* gid - identificador geográfico do município
* UF - unidade federativa
* nome - nome do município
* Censo - ano do censo relativo aos dados
* PIB - total do PIB
* Pop_est_2009 - populacao estimada	
* PIB_percapita - PIB per capita segundo os dados
* Descrição - Descrição do dados
* classe - classe do município
* desemprego - índice de desemprego na cidade no ano do Censo


### Questão 1)

Verifique a distribuição dos valores e tipos dos atributos e realize um tratamento de dados considerando:

1. Correção dos dados que for possível inferir o valor verdadeiro, ajustando e padronizando-os. Anote em quais variáveis isso ocorreu. Valores outliers não devem ser removidos/alterados, apenas dados claramente errôneos ou não padronizados.
2. Conversão dos atributos que deveriam ser numéricos para numérico - inspecione os valores para garantir que a conversão não vá gerar dados faltantes de forma desnecessária, substituindo por numeros os que forem possíveis como por exemplo o atributo "floor" como visto na aula em que substituímos dados por 0. Anote as variáveis em que isso ocorreu.
    
Quais variáveis possuiam valores que precisaram ser padronizados ou corrigidos, e quantos valores em cada uma delas foi possível corrigir?
 
(a) UF (5) e Censo (2)<br>
(b) UF (9), Desemprego (10), Censo (3)<br>
(c) Censo (2)<br>
<font color='red'>(d) UF (9) e Censo (2)
<br></font>

In [2]:
data.head()

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
0,215,BAHIA,Tremedal,2010,5.788399e+04,18433,3140.239990,Produto Interno Bruto per capita,1,9.3
1,306,RIO GRANDE DO SUL,Turuçu,2010,4.572388e+04,4000,11430.969727,Produto Interno Bruto per capita,2,4.9
2,900,ESPIRITO SANTO,Vitória,2010,1.978263e+07,320156,61790.589844,Produto Interno Bruto per capita,4,8.3
3,3613,MINAS GERAIS,Jacutinga,2010,2.478160e+05,21424,11567.209961,Produto Interno Bruto per capita,2,5.0
4,1028,PIAUÍ,Nazária,2010,2.095138e+04,7895,2653.750000,Produto Interno Bruto per capita,1,6.8


In [3]:
data.shape

(5572, 10)

In [4]:
def verifica_atributos(df):
    for var in df:
        # imprime variavel e seu tipo
        print(var,":", df[var].dtype.name, end="")
        # se nao numérico
        if not np.issubdtype(df[var].dtype, np.number):
            print("\n\t",df[var].nunique(), "distintos: ", end="")
            for val in df[var].unique():
                print("\t\t", val)
            
        else:
            print(", intervalo: ",end="")
            print(df[var].min(), ",", df[var].max())

verifica_atributos(data)

gid : int64, intervalo: 1 , 5568
UF : object
	 33 distintos: 		 BAHIA
		 RIO GRANDE DO SUL
		 ESPIRITO SANTO
		 MINAS GERAIS
		 PIAUÍ
		 GOIÁS
		 PERNAMBUCO
		 PARÁ
		 SERGIPE
		 SÃO PAULO
		 PARAÍBA
		 TOCANTINS
		 AMAZONAS
		 SANTA CATARINA
		 CEARÁ
		 RORAIMA
		 MARANHÃO
		 RIO DE JANEIRO
		 MATO GROSSO
		 PARANÁ
		 ALAGOAS
		 MATO GROSSO DO SUL
		 ACRE
		 RIO GRANDE DO NORTE
		 RONDÔNIA
		 PARANA
		 AMAPÁ
		 DISTRITO FEDERAL
		 MINAS G.
		 R.G. DO SUL
		 SP
		 PIAU
		 PARANA 
nome : object
	 5292 distintos: 		 Tremedal
		 Turuçu
		 Vitória
		 Jacutinga
		 Nazária
		 Pedra Azul
		 Teresina
		 Amarante
		 Matrinchã
		 Cachoeirinha
		 Afuá
		 Fernando de Noronha
		 Frederico Westphalen
		 Amparo de São Francisco
		 Itapeva
		 Bayeux
		 Ananás
		 Santo Antônio do Retiro
		 Serra Branca
		 Envira
		 Oliveira de Fátima
		 Elisiário
		 Otacílio Costa
		 Sericita
		 Diamantina
		 Maetinga
		 Iracema
		 São João da Baliza
		 Paulo Bento
		 Alagoinha
		 Inhuma
		 Joselândia
		 Itaguaí
		 Ced

		 Gonçalves Dias
		 Alfredo Marcondes
		 Anajás
		 Divisa Alegre
		 Guaratinga
		 Serra Azul
		 Cajazeirinhas
		 Socorro do Piauí
		 Nova Marilândia
		 Palestina
		 São José dos Ramos
		 Ererê
		 Maracás
		 Montezuma
		 Monte das Gameleiras
		 Ourém
		 Nantes
		 Campos do Jordão
		 Paraíso
		 São João da Ponte
		 José de Freitas
		 Pesqueira
		 Água Limpa
		 Cromínia
		 Tonantins
		 Canguaretama
		 Biguaçu
		 Lafaiete Coutinho
		 José Raydan
		 Maripá de Minas
		 Cajazeiras do Piauí
		 Tubarão
		 União Paulista
		 Canela
		 Divisa Nova
		 Paraíso do Norte
		 Oliveira
		 Taperoá
		 Ipuã
		 Cândido de Abreu
		 Cristalina
		 Macaíba
		 São Miguel do Iguaçu
		 Major Sales
		 Estiva
		 Bandeira do Sul
		 Cosmorama
		 Feira Nova
		 Penha
		 Santa Rosa do Purus
		 Mirassolândia
		 Currais
		 Monte do Carmo
		 Palestina de Goiás
		 Marmelópolis
		 Santo Antônio da Platina
		 Fazenda Vilanova
		 Ribeirão
		 Cambé
		 Uruoca
		 Vargeão
		 São Miguel Arcanjo
		 Cachoeiro de Itapemirim
		 São Mart

		 Redenção
		 Ibiá
		 Capetinga
		 Japonvar
		 Quintana
		 Planura
		 Pio Ix
		 Japaraíba
		 Diamante do Sul
		 Palmelo
		 Itajaí
		 Bom Jesus da Penha
		 São Miguel das Matas
		 Tapera
		 Oriximiná
		 Ministro Andreazza
		 Olho D'Água das Cunhãs
		 Riachuelo
		 Porto Lucena
		 Treze de Maio
		 Gavião
		 São José do Vale do Rio Preto
		 Ibicaré
		 Lagoa da Prata
		 Porto Mauá
		 Aquiraz
		 Morrinhos do Sul
		 Patis
		 Lauro de Freitas
		 Honório Serpa
		 Boa Vista do Buricá
		 Ipeúna
		 Botumirim
		 Serra Alta
		 Itabuna
		 Progresso
		 São Domingos das Dores
		 Inhumas
		 Itapagé
		 Carlos Chagas
		 Frei Inocêncio
		 Araguainha
		 Chã de Alegria
		 Diorama
		 Balneário Barra do Sul
		 Muliterno
		 Alcântaras
		 Fernando Prestes
		 São João do Ivaí
		 Nova Friburgo
		 Casimiro de Abreu
		 Itaqui
		 Caraá
		 Pimenteiras
		 Feliz Deserto
		 São João do Tigre
		 Ponte Alta
		 Canarana
		 Nova Odessa
		 Porto Grande
		 Quirinópolis
		 Rio das Pedras
		 Muquém de São Francisco
		 Conceição

		 Arataca
		 Macapá
		 Frei Paulo
		 Bom Princípio
		 Paraguaçu Paulista
		 Morro do Pilar
		 Alto Bela Vista
		 Frutuoso Gomes
		 Corbélia
		 Palmeirais
		 Barreirinha
		 Claro dos Poções
		 Guapiara
		 Cedro de São João
		 Brasil Novo
		 Chorrochó
		 Senador José Bento
		 Barueri
		 Palotina
		 Campo Bonito
		 Jaçanã
		 Pindoba
		 Marquinho
		 Poxoréo
		 Paranapoema
		 Jampruca
		 Serra Nova Dourada
		 Lagarto
		 Marcelândia
		 Guarantã do Norte
		 Cabo Verde
		 Ipixuna do Pará
		 São Miguel do Gostoso
		 Piraúba
		 Reserva
		 Paty do Alferes
		 Alto Garças
		 Água Azul do Norte
		 Vitória do Xingu
		 Castilho
		 Camocim
		 São José da Barra
		 José Gonçalves de Minas
		 Quaraí
		 Turilândia
		 Cacimbas
		 Lindóia do Sul
		 Arame
		 Timbaúba
		 São Felipe
		 Bandeirantes do Tocantins
		 Tocantins
		 Assaí
		 São Domingos do Prata
		 Andradina
		 Palmeirante
		 Santa Rita do Itueto
		 David Canabarro
		 Logradouro
		 São José de Ubá
		 Lagoa do Barro do Piauí
		 Pirenópolis
		 Carrap

In [5]:
data.dtypes

gid                int64
UF                object
nome              object
Censo             object
PIB              float64
Pop_est_2009       int64
PIB_percapita    float64
Descrição         object
classe             int64
desemprego       float64
dtype: object

In [6]:
data['UF'].unique()

array(['BAHIA', 'RIO GRANDE DO SUL', 'ESPIRITO SANTO', 'MINAS GERAIS',
       'PIAUÍ', 'GOIÁS', 'PERNAMBUCO', 'PARÁ', 'SERGIPE', 'SÃO PAULO',
       'PARAÍBA', 'TOCANTINS', 'AMAZONAS', 'SANTA CATARINA', 'CEARÁ',
       'RORAIMA', 'MARANHÃO', 'RIO DE JANEIRO', 'MATO GROSSO', 'PARANÁ',
       'ALAGOAS', 'MATO GROSSO DO SUL', 'ACRE', 'RIO GRANDE DO NORTE',
       'RONDÔNIA', 'PARANA', 'AMAPÁ', 'DISTRITO FEDERAL', 'MINAS G.',
       'R.G. DO SUL', 'SP', 'PIAU', 'PARANA '], dtype=object)

In [7]:
data['UF'].nunique()

33

In [8]:
# removendo os espaços
data['UF'] = data.loc[:,'UF'].str.replace(' ', '')
data['UF'].nunique()

32

In [9]:
# substituir valores para padrão
subst_UF = np.sum(
    (data['UF']=='PARANA') |
    (data['UF']=='MINASG.') |
    (data['UF']=='R.G.DOSUL') |
    (data['UF']=='SP') |
    (data['UF']=='PIAU')
    )

data.loc[data['UF']=='PARANA','UF'] = 'PARANÁ'
data.loc[data['UF']=='MINASG.','UF'] = 'MINASGERAIS'
data.loc[data['UF']=='R.G.DOSUL','UF'] = 'RIOGRANDEDOSUL'
data.loc[data['UF']=='SP','UF'] = 'SÃOPAULO'
data.loc[data['UF']=='PIAU','UF'] = 'PIAUÍ'
data['UF'].nunique()

27

In [10]:
subst_UF

9

In [11]:
data['Censo'].unique()

array(['2010', '“2010', nan, '2007', '20100'], dtype=object)

In [12]:
data['Censo'].value_counts()

2010     5559
2007        5
“2010       1
20100       1
Name: Censo, dtype: int64

In [13]:
data[data['Censo'].isna()]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
843,2192,BAHIA,Lamarão,NaN,30241.320312,12995,2327.149902,Produto Interno Bruto per capita,1,9.9
2347,3474,PERNAMBUCO,Pedra,NaN,121027.789062,20788,5822.000000,Produto Interno Bruto per capita,1,8.5
2683,5129,PERNAMBUCO,Buíque,NaN,251538.109375,53272,4721.770020,Produto Interno Bruto per capita,1,9.4
3361,1754,SÃOPAULO,São Pedro,NaN,348302.531250,31575,11030.959961,Produto Interno Bruto per capita,2,5.7
3384,5242,PERNAMBUCO,NaN,NaN,86340.601562,19026,4538.029785,Produto Interno Bruto per capita,1,9.1
3508,1767,SÃOPAULO,Salto de Pirapora,NaN,479193.375000,39616,12095.959961,Produto Interno Bruto per capita,2,7.0


In [14]:
# substituir valores para padrão
subst_Censo = np.sum(
    (data['Censo']=='“2010') |
    (data['Censo']=='20100')
    )

data.loc[data['Censo']=='“2010','Censo'] = '2010'
data.loc[data['Censo']=='20100','Censo'] = '2010'

In [15]:
data['Censo'].value_counts()

2010    5561
2007       5
Name: Censo, dtype: int64

In [16]:
subst_Censo

2

In [17]:
data['desemprego'].unique()

array([  9.3,   4.9,   8.3,   5. ,   6.8,   5.1,   4.8,   5.6,   3.9,
         7.7,   6.9,   5.3,   5.9,   9.8,   6.6,   6. ,   9.7,   8.8,
         7.9,   6.4,   4.7,   nan,  10.1,   5.5,   6.5,   7. ,   7.8,
         9.5,   8.7,   6.7,   4.1,   4. ,   9.9,   8.1,  10.3,   8.2,
         9.4,   5.2,   7.6,   8.6,   6.2,   7.2,   7.1,   9.1,   4.6,
         5.4,   3.8,   5.8,   7.3,   1. ,   3.3,   9. ,   5.7,  10.2,
         8.4,   8.5,   8.9,   9.6,  11. ,   3.6,   2.1,  10. ,   4.3,
         8. ,   6.1,   3.7,  10.8,   4.2,   4.4,   6.3,   3.4,   7.4,
         3.5,   9.2, 407. ,  12.2,   3. ,  90. ,   4.5,  10.6,   7.5,
        13.7,   3.2,  12.6,  10.7,  10.4,  11.8,  10.5,   2.3,  13.6,
         2.7,   2.2,   2.5,   2.4,   2. ,  12.3,  14. ,  12.5,  12. ,
         2.9,   1.9,  13. ,   3.1,   2.6,  11.9,  12.9,   2.8,  12.1,
        11.1])

In [18]:
data[(data['desemprego'] == 407.) | (data['desemprego'] == 90.)]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
376,1588,MARANHÃO,Carutapera,2010,67091.468750,21121,3176.530029,Produto Interno Bruto per capita,1,407.0
425,2871,PERNAMBUCO,Iguaraci,2010,45478.789062,12397,3668.530029,Produto Interno Bruto per capita,1,90.0


In [19]:
data[(data['nome'] == 'Carutapera') | (data['nome'] == 'Iguaraci')]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
376,1588,MARANHÃO,Carutapera,2010,67091.468750,21121,3176.530029,Produto Interno Bruto per capita,1,407.0
425,2871,PERNAMBUCO,Iguaraci,2010,45478.789062,12397,3668.530029,Produto Interno Bruto per capita,1,90.0


In [20]:
data[data['desemprego'].isna()]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
24,3352,MINASGERAIS,Diamantina,2010,2.876320e+05,46372,6202.709961,Produto Interno Bruto per capita,1,NaN
67,2351,PARAÍBA,Cabedelo,2010,2.332828e+06,51865,44978.851562,Produto Interno Bruto per capita,3,NaN
110,4511,BAHIA,Itaparica,2010,1.025866e+05,20796,4933.000000,Produto Interno Bruto per capita,1,NaN
140,4217,SÃOPAULO,Bastos,2010,3.223173e+05,21380,15075.650391,Produto Interno Bruto per capita,2,NaN
153,1671,RIOGRANDEDONORTE,Pedra Preta,2010,1.263997e+04,2718,4650.470215,Produto Interno Bruto per capita,1,NaN
...,...,...,...,...,...,...,...,...,...,...
5271,4453,BAHIA,Formosa do Rio Preto,2010,5.336632e+05,22171,24070.320312,Produto Interno Bruto per capita,3,NaN
5314,456,RIOGRANDEDOSUL,Guaíba,2010,1.945150e+06,96603,20135.500000,Produto Interno Bruto per capita,2,NaN
5318,1083,GOIÁS,Alexânia,2010,3.043618e+05,20706,14699.209961,Produto Interno Bruto per capita,2,NaN
5428,4547,SÃOPAULO,Itapetininga,2010,2.164772e+06,148808,14547.419922,Produto Interno Bruto per capita,2,NaN


Apesar do índice de desemprego de valor 407 estar fora do range possível (0 a 100%) e do valor 90 estar longe do valor típico (próximo a 10%) não é possível inferir quais seriam os valores corretos, portanto, nenhuma substituição deve ser feita.

In [21]:
print('Resposta:', subst_UF, 'Substituições em UF e', subst_Censo, 'Substituições em Censo')

Resposta: 9 Substituições em UF e 2 Substituições em Censo


### Questão 2)

Após o tratamento feito na Questão 2:
1. Remova colunas que possuam valores redundantes constantes
2. Exiba, e posteriormente remova municípios duplicados (considere nome e UF para esse fim), mantendo a primeira ocorrência. 

Qual o tamanho final da base de dados após esse tratamento?

(a) 5572 x 10<br>
(b) 5572 x 9<br>
(c) 5565 x 8<br>
<font color='red'>(d) 5565 x 9<br></font>

---

In [22]:
# Atributo 'Descrição' com um único valor distinto será removido
data[data['Descrição'].isna()]
data['Descrição'].unique()

array(['Produto Interno Bruto per capita'], dtype=object)

In [23]:
# Atributo 'PIB_percapita' é redundante à divisão de 'PIB' por 'Pop_est_2009'
# Antes vamos analisar se não perderemos nenhuma info útil caso fosse removido
data[['PIB', 'Pop_est_2009', 'PIB_percapita']].isna().sum()


PIB              1
Pop_est_2009     0
PIB_percapita    4
dtype: int64

In [24]:
data[data['PIB'].isna()]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
5471,549,RIOGRANDEDOSUL,NaN,2010,NaN,5732,18192.169922,Produto Interno Bruto per capita,2,5.6


In [25]:
# vamos inputar o valor de PIB para a instância 5471 para possibilitar remover o atributo 'PIB_percapita'
valor = data.iloc[5471]['PIB_percapita'] * data.iloc[5471]['Pop_est_2009'] / 1000
data.loc[data['PIB'].isna(),'PIB'] = valor
data.iloc[5471]

gid                                           549
UF                                 RIOGRANDEDOSUL
nome                                          NaN
Censo                                        2010
PIB                                 104277.517992
Pop_est_2009                                 5732
PIB_percapita                        18192.169922
Descrição        Produto Interno Bruto per capita
classe                                          2
desemprego                                    5.6
Name: 5471, dtype: object

In [26]:
# procurando por duplicatas da combinação cidade, UF
data[data[['nome', 'UF']].duplicated(keep=False)]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
2347,3474,PERNAMBUCO,Pedra,NaN,121027.789062,20788,5822.000000,Produto Interno Bruto per capita,1,8.5
2348,3474,PERNAMBUCO,Pedra,2010,121027.789062,20788,5822.000000,Produto Interno Bruto per capita,1,8.5
5461,3773,SÃOPAULO,Guapiara,2010,175721.890625,20927,8396.900391,Produto Interno Bruto per capita,1,7.2
5462,3773,SÃOPAULO,Guapiara,2010,175721.890625,20927,8396.900391,Produto Interno Bruto per capita,1,7.2
5519,1115,GOIÁS,Pirenópolis,2010,182077.437500,20945,8693.120117,Produto Interno Bruto per capita,1,4.2
5527,4655,SÃOPAULO,Mineiros do Tietê,2010,98549.375000,12334,7990.060059,Produto Interno Bruto per capita,1,8.1
5554,2102,PARÁ,Santa Maria do Pará,2010,99888.148438,23202,4305.149902,Produto Interno Bruto per capita,1,6.1
5567,2102,PARÁ,Santa Maria do Pará,2010,99888.148438,23202,4305.149902,Produto Interno Bruto per capita,1,6.1
5568,4655,SÃOPAULO,Mineiros do Tietê,2010,98549.375000,12334,7990.060059,Produto Interno Bruto per capita,1,8.1
5569,1115,GOIÁS,Pirenópolis,2010,182077.437500,20945,8693.120117,Produto Interno Bruto per capita,1,4.2


In [27]:
# removendo as duplicatas
data.drop_duplicates(keep='first', inplace=True)

In [28]:
# removendo manualmente a duplicata da linha 2347
data.drop(labels=2347, axis=0, inplace=True )
data[data['gid']==3474]

,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,Descrição,classe,desemprego
2348,3474,PERNAMBUCO,Pedra,2010,121027.789062,20788,5822.0,Produto Interno Bruto per capita,1,8.5


In [29]:
# removendo 'Descrição'
# 1PIB_percapita' não será removido pois o exercício pede remover apenas os redundantes e constantes
del data['Descrição']

In [30]:
data.shape

(5565, 9)

In [31]:
print('Resposta: Removido o atributo "Descrição", eliminadas 7 duplicatas,')
print('e o tamanho da base de dados final ficou:', data.shape, 'Linhas X Colunas')

Resposta: Removido o atributo "Descrição", eliminadas 7 duplicatas,
e o tamanho da base de dados final ficou: (5565, 9) Linhas X Colunas


---
### Questão 3)

Vamos analisar possíveis outliers. Utilize o método da análise da dispersão pelo *desvio padrão* e inspecione as colunas 'gid', 'PIB', 'Pop_est_2009', 'desemprego', procurando por outliers globais com critério de 3 desvios padrões, i.e. $3\sigma$. Nessa questão não remova outliers da base de dados, apenas identifique-os.

Quantos outliers foram encontrados, respectivamente, para 'gid', 'PIB', 'Pop_est_2009' e 'desemprego'?

(a) 0, 27, 1, 2<br>
(c) 0, 9, 44, 0<br>
(d) 1, 5, 9, 44<br>
<font color='red'>(d) 0, 5, 27, 2<br></font>


In [32]:
def remove_outliers_std(df, attributes, t):
    """Funcao para remover outliers com base no Desvio-Padrão
    Parametros:
        - df : dataframe
        - attributes: atributos a considerar na remoção
        - t factor: fator multiplicador para o desvio-padrão
    Retorno:
        dataframe com os outliers removidos
    """
    dfn = df.copy()
        
    for var in attributes:
        # verifica se variável é numerica
        if np.issubdtype(df[var].dtype, np.number):
            desvp = dfn[var].std()
            media = dfn[var].mean()
            # qtidade de outliers
            tot_out = np.sum((df[var] < media-(desvp*t)) | (df[var] > media+(desvp*t)))             
            # apenas inliers segundo std
            dfn = dfn.loc[(df[var] >= media-(desvp*t)) & (df[var] <= media+(desvp*t)),:]
            print('%s, mu = %.2f, std = %.2f, outliers = %d' % (var, media, desvp, tot_out))

    return dfn

In [33]:
# remoção dos outliers pela função criada
data_out = remove_outliers_std(df=data, attributes=['gid'], t=3)
data_out = remove_outliers_std(df=data, attributes=['PIB'], t=3)
data_out = remove_outliers_std(df=data, attributes=['Pop_est_2009'], t=3)
data_out = remove_outliers_std(df=data, attributes=['desemprego'], t=3)


gid, mu = 2784.90, std = 1607.03, outliers = 0
PIB, mu = 861901.34, std = 17671458.65, outliers = 5
Pop_est_2009, mu = 34408.02, std = 201803.45, outliers = 27
desemprego, mu = 6.67, std = 5.81, outliers = 2


Resposta: 0, 5, 27, 2 outliers

---

### Questão 4)

Utilize a base de dados após o tratamento inicial, e sem remoção de outliers. Imprima o total de valores faltantes em cada variável e, posteriormente, utilize o preenchimento por média condicionada, preenchendo 'desemprego' com as médias agrupadas por 'UF'.

Para isso codifique uma função que deverá:
1. calcular a média de uma variável alvo A (a ser preenchida) relativa a (ou agrupada por) cada valor distinto da variável que se deseja usar para agrupar;
2. atribuir a média calculada de forma agrupada a todas as linhas cuja variável alvo é faltante e que possua o valor da variável categórica correspondente;
3. o valor atribuido deve seguir o mesmo tipo da variável alvo, ou seja, int, float, etc. Quando int, realize o arredondamento utlizando `np.round(,0)`, quando float64 utilize `np.round(,1)`

Quantos dados faltantes existiam em "desemprego", e qual a média e o desvio padrão dessa variável após o preenchimento?

<font color='red'>(a) 62 faltantes, média 6.6664, desvio padrão 5.7834<br></font>
(b) 59 faltantes, média 6.6663, desvio padrão 5.7834<br>
(c) 62 faltantes, média 5.8126, desvio padrão 5.7834<br>
(d) 62 faltantes, média 6.6663, desvio padrão 4.5120<br>


In [34]:
desemprego_nan = data['desemprego'].isna().sum()
data.isna().sum()

gid               0
UF                0
nome              2
Censo             5
PIB               0
Pop_est_2009      0
PIB_percapita     4
classe            0
desemprego       62
dtype: int64

In [35]:
def missing_condmean(df, att, att_cat):
    """Funcao para preencher faltantes de variáveis numéricas utilizando:
    a média condicionada a outra variável categórica (não numérica)
    Parametros:
        - df : dataframe
        - att: atributo numérico com faltantes a serem preenchidos
        - att_cat: atributo categórico condicional à qual os faltantes serão preenchidos
    Retorno:
        dataframe com os faltantes preenchidos
    """
        
    dfn = df.copy()
    print('- preencher ', att, ' condicionado a ', att_cat, end=' : ')
    nullatt = dfn[att].isnull()                                            # encontra os faltantes no att numérico
    print(np.sum(nullatt), 'faltantes')                                    # soma dos faltantes

    for j in df[att_cat].unique():                                         # j é cada valor distinto do att categórico
        mu_cat = np.round(np.mean(df.loc[df[att_cat]==j, att]),1)          # média de att para att_cat igual a j, arred=1
        
        if (df[att].dtypes == np.int64):                                   # se dtype da att for um número inteiro
            mu_cat = np.round(mu_cat,0)                                    # média de att é arredondada para 0 decimais
            
        print('\t %s media = %.1f' % (j, mu_cat))                          # print do valor att_cat e da média
        dfn.loc[(df[att].isnull()) & (df[att_cat]==j), att] = mu_cat       # preenche os faltantes com base no valor j
        
    return dfn

In [36]:
data_fill = missing_condmean(df=data, att='desemprego', att_cat='UF')

- preencher  desemprego  condicionado a  UF : 62 faltantes
	 BAHIA media = 9.1
	 RIOGRANDEDOSUL media = 4.5
	 ESPIRITOSANTO media = 7.6
	 MINASGERAIS media = 5.9
	 PIAUÍ media = 5.8
	 GOIÁS media = 4.7
	 PERNAMBUCO media = 9.1
	 PARÁ media = 7.2
	 SERGIPE media = 8.4
	 SÃOPAULO media = 7.0
	 PARAÍBA media = 9.0
	 TOCANTINS media = 5.9
	 AMAZONAS media = 8.1
	 SANTACATARINA media = 6.8
	 CEARÁ media = 5.3
	 RORAIMA media = 7.6
	 MARANHÃO media = 8.3
	 RIODEJANEIRO media = 8.2
	 MATOGROSSO media = 7.2
	 PARANÁ media = 4.5
	 ALAGOAS media = 9.4
	 MATOGROSSODOSUL media = 4.5
	 ACRE media = 4.1
	 RIOGRANDEDONORTE media = 9.6
	 RONDÔNIA media = 3.2
	 AMAPÁ media = 12.7
	 DISTRITOFEDERAL media = 8.1


In [37]:
print('faltantes em desemprego=', desemprego_nan)
print('--------------------------------------------')
print('media antes do preenchimento =', np.round(np.mean(data['desemprego']), 4))
print('desvio-padrão antes do preenchimento =', np.round(np.std(data['desemprego']), 4))
print('--------------------------------------------')
print('media após o preenchimento =', np.round(np.mean(data_fill['desemprego']), 4))
print('desvio-padrão após o preenchimento =', np.round(np.std(data_fill['desemprego']), 4))

faltantes em desemprego= 62
--------------------------------------------
media antes do preenchimento = 6.6663
desvio-padrão antes do preenchimento = 5.813
--------------------------------------------
media após o preenchimento = 6.6664
desvio-padrão após o preenchimento = 5.7829


In [38]:
print('faltantes em desemprego=', desemprego_nan)
print('--------------------------------------------')
print('media antes do preenchimento =', np.round((data['desemprego']).mean(), 4))
print('desvio-padrão antes do preenchimento =', np.round((data['desemprego']).std(), 4))
print('--------------------------------------------')
print('media após o preenchimento =', np.round((data_fill['desemprego']).mean(), 4))
print('desvio-padrão após o preenchimento =', np.round((data_fill['desemprego']).std(), 4))

faltantes em desemprego= 62
--------------------------------------------
media antes do preenchimento = 6.6663
desvio-padrão antes do preenchimento = 5.8136
--------------------------------------------
media após o preenchimento = 6.6664
desvio-padrão após o preenchimento = 5.7834


---

### Questão 5)

Desejamos projetar um algoritmo de aprendizado em que o atributo alvo é 'classe', inicialmente utilizando apenas dados da UF Paraná, mas posteriormente deveremos utilizar o modelo aprendido em toda a base de dados. 

Use os dados tratados após o preenchimento de dados faltantes de desemprego, porém sem remoção de outliers. Considere o atributo 'classe' e analise a distribuição dos seus valores em dois cenários:
1. base completa
2. considerando apenas a UF 'Paraná'. 

Após a análise podemos afirmar que:

(a) Os cenários 1 e 2 são desbalanceados, com classes minoritárias diferentes e majoritárias iguais.<br>
(b) Os cenários 1 e 2 são desbalanceados, com distribuição de classes similar.<br>
(c) Apenas o cenário 2 é desbalanceado. <br>
<font color='red'>(d) Os cenários 1 e 2 são desbalanceados, com classes majoritárias diferentes.<br></font>

In [39]:
# base completa
data_fill['classe'].value_counts()

1    3156
2    1977
3     376
4      51
5       5
Name: classe, dtype: int64

In [40]:
data_fill['classe'].value_counts(normalize=True)

1    0.567116
2    0.355256
3    0.067565
4    0.009164
5    0.000898
Name: classe, dtype: float64

In [41]:
# base UF = Paraná
(data_fill['UF']=='PARANÁ').sum()

399

In [42]:
data[data_fill['UF']=='PARANÁ']['classe'].value_counts()

2    237
1    140
3     20
4      2
Name: classe, dtype: int64

In [43]:
data[data_fill['UF']=='PARANÁ']['classe'].value_counts(normalize=True)

2    0.593985
1    0.350877
3    0.050125
4    0.005013
Name: classe, dtype: float64

Ambos os cenários são desbalanceados.   
As classes minoritárias 3, 4 e 5 se mantém.   
As classes majoritárias 1 e 2 se invertem entre os cenários.   
Portanto, não são iguais, são diferentes.